In [3]:
# =========================
# Imports
# =========================
import os, random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast as autocast_ctx, GradScaler

from torchvision import models, transforms

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from PIL import Image
from tqdm import tqdm

In [4]:
# --- Reproducibility ---
SEED = 1337
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
# --- Device / AMP ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
scaler = GradScaler() if device.type == 'cuda' else None

/tmp/ipython-input-710767593.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == 'cuda' else None


In [6]:
# --- Paths ---
BASE_DIR   = Path(".")
IMAGES_DIR = BASE_DIR / "Images"
SUB_DIR    = BASE_DIR / "submissions"
SUB_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
# --- Hyperparams / Config ---
BACKBONE       = "resnet34"    # or "resnet18"
IMG_SIZE       = 320           # 256 for r18 baseline
FOLDS          = 5
EPOCHS         = 15            # adjust if needed
WARMUP_EPOCHS  = 1
LR             = 3e-4
WEIGHT_DECAY   = 1e-4
BATCH_SIZE     = 32
NUM_WORKERS    = 0             # 0 is safe on Colab/Windows
PIN_MEMORY     = (device.type == 'cuda')
SMOOTH_EPS     = 0.05

In [15]:
# === Setup: Clone repo & set base directory ===
# If you haven't cloned in this runtime yet, run:
!git clone https://github.com/Gaabshiine/pycon-2025-hackthon.git
BASE_DIR = Path('/content/pycon-2025-hackthon')

fatal: destination path 'pycon-2025-hackthon' already exists and is not an empty directory.


In [16]:
# --- DataFrames ---
# Expect Train.csv with columns: Image_id, Label
#       Test.csv  with column:   Image_id
train_df = pd.read_csv(BASE_DIR / "Train.csv")
test_df  = pd.read_csv(BASE_DIR / "Test.csv")

In [17]:
# (Optional) quick sanity
assert "Image_id" in train_df.columns and "Label" in train_df.columns
assert "Image_id" in test_df.columns
assert IMAGES_DIR.exists(), f"Missing folder: {IMAGES_DIR}"

In [18]:
# =========================
# 0) (Optional) minimal installs if running in a very bare Colab (you already have requirements.txt)
# =========================
# !pip -q install -U tqdm pillow scikit-learn

# =========================
# 1) (Optional) Quick EDA: class balance
# =========================
# train_df['Label'].value_counts().sort_index().plot(
#     kind="bar", title="Class balance (0=healthy, 1=fall armyworm)"
# )
# plt.show()

In [19]:
# =========================
# 2) Dataset & transforms
# =========================
class MaizeDataset(Dataset):
    def __init__(self, df, images_dir, mode='train', transform=None):
        self.df = df.reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.images_dir / row['Image_id']  # CSV already has .jpg
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.mode == 'test':
            return img, row['Image_id']
        else:
            label = torch.tensor(float(row['Label']), dtype=torch.float32)
            return img, label

# Transforms for resnet34 @ 320px
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

valid_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE + 32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

In [20]:
# =========================
# 3) Model, loss, scheduler, early stopping
# =========================
def build_model():
    if BACKBONE == "resnet34":
        m = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
    else:
        m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, 1)
    return m

def bce_with_logits_smooth(logits, labels, eps=SMOOTH_EPS):
    # Smooth labels toward 0.5 to regularize
    labels_s = labels * (1 - eps) + 0.5 * eps
    return F.binary_cross_entropy_with_logits(logits, labels_s)

def make_scheduler(optimizer):
    def lr_lambda(e):
        if e < WARMUP_EPOCHS:      # linear warmup
            return (e + 1) / max(1, WARMUP_EPOCHS)
        t = (e - WARMUP_EPOCHS) / max(1, (EPOCHS - WARMUP_EPOCHS))
        return 0.5 * (1 + np.cos(np.pi * t))  # cosine decay
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

class EarlyStopper:
    def __init__(self, patience=3):
        self.best = -np.inf
        self.wait = 0
        self.patience = patience
    def step(self, val):
        if val > self.best + 1e-4:
            self.best = val
            self.wait = 0
            return True
        self.wait += 1
        return False
    def should_stop(self):
        return self.wait >= self.patience

In [21]:
# =========================
# 4) Train / Eval loops
# =========================
def train_one_epoch(model, loader, optimizer, scaler):
    model.train()
    total = 0.0
    for imgs, labels in tqdm(loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        optimizer.zero_grad(set_to_none=True)
        with autocast_ctx(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(imgs)
            loss = bce_with_logits_smooth(logits, labels)
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total += loss.item() * imgs.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_probs, all_targets = [], []
    for imgs, labels in loader:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1)
        with autocast_ctx(device_type='cuda', enabled=(device.type=='cuda')):
            probs = torch.sigmoid(model(imgs))
        all_probs.append(probs.cpu().numpy())
        all_targets.append(labels.cpu().numpy())
    probs = np.concatenate(all_probs).ravel()
    targs = np.concatenate(all_targets).ravel()
    return roc_auc_score(targs, probs)

In [22]:
# =========================
# 5) TTA inference helper
# =========================
@torch.no_grad()
def predict_tta(model, imgs):
    # orig + hflip + vflip (×3)
    with autocast_ctx(device_type='cuda', enabled=(device.type=='cuda')):
        logits  = model(imgs)
        logits += model(torch.flip(imgs, [3]))  # horizontal flip
        logits += model(torch.flip(imgs, [2]))  # vertical flip
    return torch.sigmoid(logits / 3.0)

In [ ]:
# =========================
# 6) 5-fold CV training + TTA inference + save submission
# =========================
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

test_ds = MaizeDataset(test_df, IMAGES_DIR, 'test', valid_tfms)
test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)

test_preds = np.zeros(len(test_df), dtype=np.float32)

for fold, (tr_idx, va_idx) in enumerate(skf.split(train_df['Image_id'], train_df['Label']), 1):
    print(f"\n========== Fold {fold}/{FOLDS} ==========")
    tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
    va_df = train_df.iloc[va_idx].reset_index(drop=True)

    tr_loader = DataLoader(
        MaizeDataset(tr_df, IMAGES_DIR, 'train', train_tfms),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
    va_loader = DataLoader(
        MaizeDataset(va_df, IMAGES_DIR, 'train', valid_tfms),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

    model = build_model().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = make_scheduler(optimizer)
    early = EarlyStopper(patience=3)

    best_path = BASE_DIR / f'best_{BACKBONE}_fold{fold}.pt'
    best_auc = -1.0

    for epoch in range(1, EPOCHS + 1):
        loss = train_one_epoch(model, tr_loader, optimizer, scaler)
        val_auc = evaluate(model, va_loader)
        print(f"Fold {fold} | Epoch {epoch:02d} | loss {loss:.4f} | val_auc {val_auc:.6f}")
        if early.step(val_auc):
            torch.save(model.state_dict(), best_path)
            best_auc = val_auc
            print("  ✓ Saved best")
        scheduler.step()
        if early.should_stop():
            print("  Early stopping.")
            break

    # Load best & predict test with TTA
    model.load_state_dict(torch.load(best_path, map_location=device))
    model.eval()
    fold_probs = []
    for imgs, _ids in tqdm(test_loader, leave=False):
        imgs = imgs.to(device, non_blocking=True)
        p = predict_tta(model, imgs).cpu().numpy().ravel()
        fold_probs.append(p)
    fold_probs = np.concatenate(fold_probs)
    test_preds += fold_probs / FOLDS
    print(f"Fold {fold} best AUC: {best_auc:.6f}")

# Save final submission with the exact required filename
sub5 = pd.DataFrame({'Image_id': test_df['Image_id'], 'Label': test_preds})
final_path = SUB_DIR / "submission_cv5_resnet34_img320.csv"
sub5.to_csv(final_path, index=False)
print("Saved:", final_path)


========== Fold 1/5 ==========
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 199MB/s]


  0%|          | 0/41 [00:00<?, ?it/s]

Fold 1 | Epoch 01 | loss 0.2119 | val_auc 0.998514
  ✓ Saved best


  0%|          | 0/41 [00:00<?, ?it/s]

...

  0%|          | 0/41 [00:00<?, ?it/s]

Fold 2 | Epoch 05 | loss 0.1381 | val_auc 1.000000


  0%|          | 0/41 [00:00<?, ?it/s]

Fold 2 | Epoch 06 | loss 0.1249 | val_auc 1.000000


  0%|          | 0/41 [00:00<?, ?it/s]